### Import des données

In [1]:
import pandas as pd
import numpy as np 

df = pd.read_csv("clean_data.csv")
df = df.drop(["age_category", "bmi_index", "sex_categ"], axis=1) # avec bmi 
df 

,age,sex,bmi,children,smoker,region,charges,bmi_categories
0,19,female,27.900,0,1,southwest,16884.92400,overweight
1,18,male,33.770,1,0,southeast,1725.55230,obesity class 1
2,28,male,33.000,3,0,southeast,4449.46200,obesity class 1
3,33,male,22.705,0,0,northwest,21984.47061,normal
4,32,male,28.880,0,0,northwest,3866.85520,overweight
...,...,...,...,...,...,...,...,...
1332,50,male,30.970,3,0,northwest,10600.54830,obesity class 1
1333,18,female,31.920,0,0,northeast,2205.98080,obesity class 1
1334,18,female,36.850,0,0,southeast,1629.83350,obesity class 2
1335,21,female,25.800,0,0,southwest,2007.94500,overweight


### Nettoyage/Encodage des données & pipelines, transformers, modèles...

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import RobustScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer 

# selection des features et de la cible
X = df.drop('charges', axis=1)
y = df.charges

# Création du train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=42)

# Sélection des variables numériques
num_col = list(X.select_dtypes(include=[float,int]).columns)
# Sélection des variables catégorielles
cat_col = list(X.select_dtypes(include=[object]).columns)


# pipeline pour les valeurs numériques
# cela permet de créer une succession d'étapes (ici il n'y en a qu'une)
my_num_pipe = make_pipeline(RobustScaler())

# ColumnTransformer permet de faire des opérations sur des sélections de colonnes
preprocessing = ColumnTransformer([
    ("one_hot", OneHotEncoder(),cat_col),
    ("scaling", my_num_pipe, num_col)
])

#### Voir si les données ont bien été modifiées

In [3]:
# Appliquez le pipeline sur les données d'entraînement et de test
X_train_transformed = preprocessing.fit_transform(X_train)
X_train_transformed

array([[ 0.        ,  1.        ,  1.        , ..., -0.68969697,
        -0.5       ,  0.        ],
       [ 0.        ,  1.        ,  1.        , ..., -0.54      ,
         0.5       ,  0.        ],
       [ 1.        ,  0.        ,  0.        , ...,  0.88787879,
         0.5       ,  0.        ],
       ...,
       [ 0.        ,  1.        ,  0.        , ..., -0.99393939,
         0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        , ..., -0.26666667,
         1.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        , ...,  0.68606061,
         0.5       ,  0.        ]])

In [4]:
# Supposez que X_train_transformed soit une matrice NumPy résultante de la transformation
# Appliquer le ColumnTransformer sur les données d'entraînement
X_train_transformed = preprocessing.fit_transform(X_train)

# Obtenez les noms de colonnes après la transformation
all_columns = preprocessing.transformers_[0][1].get_feature_names_out(cat_col).tolist() + num_col

# Créez un DataFrame pandas à partir des données transformées et des noms de colonnes
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=all_columns)

# Affichez le DataFrame
print(X_train_transformed_df.head())

   sex_female  sex_male  region_northeast  region_northwest  region_southeast  \
0         0.0       1.0               1.0               0.0               0.0   
1         0.0       1.0               1.0               0.0               0.0   
2         1.0       0.0               0.0               1.0               0.0   
3         0.0       1.0               0.0               0.0               1.0   
4         1.0       0.0               1.0               0.0               0.0   

   region_southwest  bmi_categories_normal  bmi_categories_obesity class 1  \
0               0.0                    1.0                             0.0   
1               0.0                    0.0                             0.0   
2               0.0                    0.0                             0.0   
3               0.0                    0.0                             0.0   
4               0.0                    0.0                             0.0   

   bmi_categories_obesity class 2  bmi_categ

### DummyRegressor

In [5]:
# DUMMY REGRESSOR (ne prend pas en compte les variables, prédit la moyenne de chaque valeur Y)
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X, y)
print(f"regression dummy : {dummy_regr.score(X_test, y_test)}")

regression dummy : -0.0053648447378846775


### LinearRegression

In [6]:
# REGRESSION LINEAIRE
# j'ajoute la régression linéaire au pipeline
my_pipe_lr = make_pipeline(preprocessing, LinearRegression())
# Je fit sur les données
my_pipe_lr.fit(X_train, y_train)
# afficher le score
print(f"regression linéaire : {my_pipe_lr.score(X_test, y_test)}")

regression linéaire : 0.801935476933986


In [7]:
my_pipe_lr[-1].coef_
# plus le coef est gros plus l'info est importante

array([1.83394418e+16, 1.83394418e+16, 9.24038430e+16, 9.24038430e+16,
       9.24038430e+16, 9.24038430e+16, 2.84361458e+15, 2.84361458e+15,
       2.84361458e+15, 2.84361458e+15, 2.84361458e+15, 2.84361458e+15,
       5.87075000e+03, 3.91500000e+02, 1.12600781e+03, 2.30612500e+04])

### Lasso

In [8]:
# REGRESSION LASSO CV
my_pipe_lasso = make_pipeline(preprocessing, Lasso(alpha=9.91))
# Je fit sur les données
my_pipe_lasso.fit(X_train, y_train)
# le score
print(f"regression lasso : {my_pipe_lasso.score(X_test, y_test)}")

regression lasso : 0.803034590646162


In [9]:
from sklearn.model_selection import GridSearchCV
param = {"lasso__alpha" : np.arange(0.01,10, 0.1)}

grid = GridSearchCV(my_pipe_lasso, param, cv=5)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('one_hot',
                                                                         OneHotEncoder(),
                                                                         ['sex',
                                                                          'region',
                                                                          'bmi_categories']),
                                                                        ('scaling',
                                                                         Pipeline(steps=[('robustscaler',
                                                                                          RobustScaler())]),
                                                                         ['age',
                                                                          'bmi',
                                                                          'children',
                                                                          'smoker'])])),
                                       ('lasso', Lasso(alpha=9.91))]),
             param_grid={'lasso__alpha': array([0.01, 0.11, 0.21, 0.31, 0.41, 0.5...
       2.21, 2.31, 2.41, 2.51, 2.61, 2.71, 2.81, 2.91, 3.01, 3.11, 3.21,
       3.31, 3.41, 3.51, 3.61, 3.71, 3.81, 3.91, 4.01, 4.11, 4.21, 4.31,
       4.41, 4.51, 4.61, 4.71, 4.81, 4.91, 5.01, 5.11, 5.21, 5.31, 5.41,
       5.51, 5.61, 5.71, 5.81, 5.91, 6.01, 6.11, 6.21, 6.31, 6.41, 6.51,
       6.61, 6.71, 6.81, 6.91, 7.01, 7.11, 7.21, 7.31, 7.41, 7.51, 7.61,
       7.71, 7.81, 7.91, 8.01, 8.11, 8.21, 8.31, 8.41, 8.51, 8.61, 8.71,
       8.81, 8.91, 9.01, 9.11, 9.21, 9.31, 9.41, 9.51, 9.61, 9.71, 9.81,
       9.91])})

In [10]:
np.arange(0.01,10, 0.1)

array([0.01, 0.11, 0.21, 0.31, 0.41, 0.51, 0.61, 0.71, 0.81, 0.91, 1.01,
       1.11, 1.21, 1.31, 1.41, 1.51, 1.61, 1.71, 1.81, 1.91, 2.01, 2.11,
       2.21, 2.31, 2.41, 2.51, 2.61, 2.71, 2.81, 2.91, 3.01, 3.11, 3.21,
       3.31, 3.41, 3.51, 3.61, 3.71, 3.81, 3.91, 4.01, 4.11, 4.21, 4.31,
       4.41, 4.51, 4.61, 4.71, 4.81, 4.91, 5.01, 5.11, 5.21, 5.31, 5.41,
       5.51, 5.61, 5.71, 5.81, 5.91, 6.01, 6.11, 6.21, 6.31, 6.41, 6.51,
       6.61, 6.71, 6.81, 6.91, 7.01, 7.11, 7.21, 7.31, 7.41, 7.51, 7.61,
       7.71, 7.81, 7.91, 8.01, 8.11, 8.21, 8.31, 8.41, 8.51, 8.61, 8.71,
       8.81, 8.91, 9.01, 9.11, 9.21, 9.31, 9.41, 9.51, 9.61, 9.71, 9.81,
       9.91])

In [11]:
grid.best_params_

{'lasso__alpha': 3.11}

### Ridge

In [12]:
# REGRESSION RIDGE CV 
my_pipe_ridge = make_pipeline(preprocessing, Ridge())
# Je fit sur les données
my_pipe_ridge.fit(X_train, y_train)
# le score
print(f"regression ridge : {my_pipe_ridge.score(X_test, y_test)}")

regression ridge : 0.8015545523258987


### ElasticNet

In [20]:
# REGRESSION ELASTICNETCV
my_pipe_elasticnet = make_pipeline(preprocessing, ElasticNet(alpha=1e-05)) # avant alpha = 0.01
# Je fit sur les données
my_pipe_elasticnet.fit(X_train, y_train)
# le score
print(f"regression elasticnet : {my_pipe_elasticnet.score(X_test, y_test)}")

regression elasticnet : 0.801951559414776


/home/utilisateur/Documents/DEV-IA/projets/machine_learning_assurance/machinelearning_assurance/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.911e+10, tolerance: 1.464e+07
  model = cd_fast.enet_coordinate_descent(


In [18]:
from sklearn.model_selection import GridSearchCV
param = {"elasticnet__alpha" : np.arange(0.00001, 0.0001, 0.1)}

grid = GridSearchCV(my_pipe_elasticnet, param, cv=5)
grid.fit(X_train, y_train)

/home/utilisateur/Documents/DEV-IA/projets/machine_learning_assurance/machinelearning_assurance/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.437e+10, tolerance: 1.138e+07
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Documents/DEV-IA/projets/machine_learning_assurance/machinelearning_assurance/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.573e+10, tolerance: 1.168e+07
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Documents/DEV-IA/projets/machine_learning_assurance/machinelearning_assurance/.venv/lib/python3.11/sit

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('one_hot',
                                                                         OneHotEncoder(),
                                                                         ['sex',
                                                                          'region',
                                                                          'bmi_categories']),
                                                                        ('scaling',
                                                                         Pipeline(steps=[('robustscaler',
                                                                                          RobustScaler())]),
                                                                         ['age',
                                                                          'bmi',
                                                                          'children',
                                                                          'smoker'])])),
                                       ('elasticnet', ElasticNet(alpha=0.01))]),
             param_grid={'elasticnet__alpha': array([1.e-05])})

In [15]:
len(np.arange(0.01, 10, 0.1))

100

In [19]:
grid.best_params_

{'elasticnet__alpha': 1e-05}